# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
# File to Load (Remember to Change These)
city_data_load = "WeatherPy/WeatherPy.csv"
#student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
city_df = pd.read_csv(city_data_load)
#student_data = pd.read_csv(student_data_to_load)
city_df.dropna(inplace = True) 
city_df

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity
0,Hofn,IS,39.18,1619751639,15,64.2539,-15.2082,6.67,62
1,Ilulissat,GL,33.80,1619751508,100,69.2167,-51.1000,4.52,75
2,Punta Arenas,CL,42.80,1619751276,90,-53.1500,-70.9167,9.22,93
3,Mar del Plata,AR,63.00,1619750963,0,-38.0023,-57.5575,11.50,82
4,Yellowknife,CA,32.00,1619751231,75,62.4560,-114.3525,11.50,47
...,...,...,...,...,...,...,...,...,...
562,Cayenne,GF,78.01,1619751963,40,4.9333,-52.3333,2.30,100
563,San Quintín,MX,62.82,1619751404,0,30.4833,-115.9500,5.93,78
564,Policoro,IT,55.99,1619751964,93,40.2115,16.6726,4.79,99
565,Masvingo,ZW,60.93,1619751964,1,-20.0637,30.8277,4.29,71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

In [28]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
city1 = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70)]
city2 = city1.loc[(city1["Wind Speed"] < 10)]
city3 = city2.loc[city2["Cloudiness"] == 0]
city3

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity
46,Awjilah,LY,78.78,1619751338,0,29.1081,21.2869,9.55,15
61,Riyadh,SA,73.40,1619751363,0,24.6877,46.7219,3.44,27
159,Baghdad,IQ,73.40,1619751613,0,33.3406,44.4009,6.91,38
237,Tikrit,IQ,77.92,1619751869,0,34.6071,43.6782,9.15,24
357,Codrington,AU,73.96,1619751900,0,-38.2667,141.9667,9.78,36
373,Aksu,CN,75.83,1619751912,0,41.1231,80.2644,5.14,11
388,Ampanihy,MG,70.29,1619751915,0,-24.7000,44.7500,6.15,70
484,Batemans Bay,AU,71.01,1619751941,0,-35.7167,150.1833,7.38,59


In [30]:
#city4 = city3.loc[(city1["Wind Speed"] > 5)]
#city4

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df = pd.DataFrame({})
hotel_df


hotel_df["Hotel Name"]= ""
hotel_df["City"] = city3["City"]
hotel_df["Country"]=city3["Country"]
hotel_df["Lat"] = ""
hotel_df["Lng"]=""


hotel_df

,Hotel Name,City,Country,Lat,Lng
46,NaN,Awjilah,LY,,
61,NaN,Riyadh,SA,,
159,NaN,Baghdad,IQ,,
237,NaN,Tikrit,IQ,,
357,NaN,Codrington,AU,,
373,NaN,Aksu,CN,,
388,NaN,Ampanihy,MG,,
484,NaN,Batemans Bay,AU,,


In [32]:

for index, row in city3.iterrows():
   

    lat = city3.loc[index,"Lat"]
    lng = city3.loc[index,"Lng"]
    target_radius = 5000
    target_type = "hotel"
    
    
    params = {
    "location": f"{lat},{lng}",
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
  
# base url
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
# run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
 
    geo = results[0]["geometry"]
    loc = geo["location"]
    lt = loc["lat"]
    ln = loc["lng"]
# Print the json (pretty printed)
    #print(json.dumps(results, indent=4, sort_keys=False))
    
    print(f"Retrieving Results for Index {index}")
    print(results[0]["name"])
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    hotel_df.loc[index, 'Lat'] = loc["lat"]
    hotel_df.loc[index, 'Lng'] = loc["lng"]
    count+=1
    print("======================")
    
        
        

Retrieving Results for Index 46
Awjilah
Retrieving Results for Index 61
Riyadh
Retrieving Results for Index 159
Baghdad
Retrieving Results for Index 237
Tikrit
Retrieving Results for Index 357
Yambuk
Retrieving Results for Index 373
Aksu Prefecture
Retrieving Results for Index 388
Ampanihy
Retrieving Results for Index 484
Batemans Bay


In [33]:
hotel_df

,Hotel Name,City,Country,Lat,Lng
46,Awjilah,Awjilah,LY,29.0906,21.3322
61,Riyadh,Riyadh,SA,24.7136,46.6753
159,Baghdad,Baghdad,IQ,33.3152,44.3661
237,Tikrit,Tikrit,IQ,34.599,43.675
357,Yambuk,Codrington,AU,-38.3081,142.041
373,Aksu Prefecture,Aksu,CN,41.1688,80.2606
388,Ampanihy,Ampanihy,MG,-24.693,44.7452
484,Batemans Bay,Batemans Bay,AU,-35.7162,150.179


In [34]:
# geocoordinates
#target_coordinates = "-9.6658,-35.7353"
#print(target_coordinates)
#target_coordinates = city3["Lat"],city3["Lng"]
#target_radius = 3000
#target_type = "hotel"

# set up a parameters dictionary
#params = {
  #  "location": target_coordinates,
   # "radius": target_radius,
 #   "type": target_type,
 #   "key": g_key
#}

# base url
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response = requests.get(base_url, params=params).json()



In [35]:
# convert response to json
#places_data = response.json()
#results = response['results']
# Print the json (pretty printed)
#print(json.dumps(results, indent=4, sort_keys=False))

In [36]:
#print(results[0]["name"])
#print(results[0]["geometry"])
#g = results[0]["geometry"]
#a = g["location"]
#b = a["lat"]
#b

In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]

# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))